In [68]:
# Librerías principales
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, LassoCV
from sklearn.model_selection import StratifiedKFold
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")


In [41]:
# Definir la ruta al archivo
ruta_dta = "C:/SEM 8/Bloque dos & materias final/Proyecto BeWay/tablas & codigos/dtas/BASE_ENSAFI.dta"

# Leer el archivo .dta
data = pd.read_stata(ruta_dta)

# Ver las primeras filas para asegurarnos
print(data.head())


   llaveviv  p1_1  p1_2  p1_3  p1_4_01  p1_4_02  p1_4_03  p1_4_04  p1_4_05  \
0  10001601     2     5     2        1        1        1        1        1   
1  10001602     3     5     1        1        1        1        1        2   
2  10001603     2     5     1        1        1        1        1        1   
3  10001604     1     5     2        1        1        1        2        1   
4  10001605     2     5     1        1        1        1        1        1   

   p1_4_06  ...  gra_control  orien_fut  impulsivid  orien_acci  optimismo  \
0        1  ...            1          2           2           2          1   
1        2  ...            1          1           2           1          1   
2        2  ...            1          2           1           1          1   
3        1  ...            2          1           2           1          2   
4        2  ...            2          2           1           1          2   

   gtos_ahorr  depen_sum  dep_eco  fac_ele  pob_objetivo  
0  

In [43]:
# Paso 1: Definir las listas de variables a conservar y eliminar

#conservmos todas las variables que empiezan por "p" y las que no empiezan por "p" pero están en la lista de conservación

# Variables x ELIMINAR (A)
variables_a_eliminar = [
    'p3_6_1', 'p3_6_2', 'p3_6_3', 'p4_2', 'p4_3', 'paren', 'p5_4a', 'p5_6', 
    'p5_19', 'p5_19a', 'p5_22', 'p6_12', 'p6_13'
]

# Variables x CONSERVAR aunque no empiecen con "p" (B)
variables_no_p_a_conservar = [
    'ent', 'tloc', 'region', 'sexo', 'edad_v', 'niv',
    'filtro_s4_1', 'filtro_s5_3', 'filtro_s5_2', 'filtro_s5_3_1', 'filtro_s5_4',
    'filtro_s6_1', 'filtro_s6_2', 'ingreso_mensual', 'gasto_mensual', 'pob_objetivo',
    'niv_bienes', 'niv_estres', 'conf_finan', 'gra_control', 'orien_fut', 'impulsivid',
    'orien_acci', 'optimismo', 'gtos_ahorr', 'fac_ele', 'depen_sum'
]

# Paso 2: Crear el nuevo DataFrame

# 1. Quedarse sólo con variables que empiecen con 'p' o estén en la lista de conservación
variables_a_conservar = [
    col for col in data.columns 
    if col.startswith('p') or col in variables_no_p_a_conservar
]

# 2. De esas variables, eliminar las que estén en la lista de eliminación
variables_finales = [
    col for col in variables_a_conservar 
    if col not in variables_a_eliminar
]

# 3. Crear la copia limpia
data_copy_clean = data[variables_finales].copy()

print(data_copy_clean.head(5))




   p1_1  p1_2  p1_3  p1_4_01  p1_4_02  p1_4_03  p1_4_04  p1_4_05  p1_4_06  \
0     2     5     2        1        1        1        1        1        1   
1     3     5     1        1        1        1        1        2        2   
2     2     5     1        1        1        1        1        1        2   
3     1     5     2        1        1        1        2        1        1   
4     2     5     1        1        1        1        1        1        2   

   p1_4_07  ...  conf_finan  gra_control  orien_fut  impulsivid  orien_acci  \
0        1  ...           1            1          2           2           2   
1        1  ...           1            1          1           2           1   
2        1  ...           1            1          2           1           1   
3        2  ...           2            2          1           2           1   
4        1  ...           2            2          2           1           1   

   optimismo  gtos_ahorr  depen_sum  fac_ele  pob_objetivo  
0

In [44]:
# Contar cuántas variables tiene el DataFrame
num_variables = data_copy_clean.shape[1]
print(f"El DataFrame tiene {num_variables} variables.")

# Ruta del archivo de salida (asumiendo estructura de carpetas mencionada)
ruta_salida = "../outputspy/vars_list.txt"

# Guardar nombres de columnas en el archivo
with open(ruta_salida, "w", encoding="utf-8") as f:
    for col in data_copy_clean.columns:
        f.write(f"{col}\n")


El DataFrame tiene 302 variables.


In [47]:
# Paso 1: Definir variables a excluir de la dummificación
variable_dependiente = 'pob_objetivo'

variables_no_dummies = [
    'p1_1', 'p1_2', 'p1_3', 'p2_1', 'p8_4',
    'edad_v', 'ingreso_mensual', 'gasto_mensual', 'fac_ele',
    variable_dependiente  # ya es binaria
]

# Paso 2: Separar variables a dummificar y conservar
variables_dummificar = [col for col in data_copy_clean.columns if col not in variables_no_dummies]
variables_conservar = variables_no_dummies

# Paso 3: Homogeneizar tipos de variables a dummificar
for col in variables_dummificar:
    nunique = data_copy_clean[col].nunique(dropna=True)
    if data_copy_clean[col].dtype in ['int64', 'float64', 'bool'] and nunique < 20:
        data_copy_clean[col] = data_copy_clean[col].astype(str)
    elif data_copy_clean[col].dtype == 'bool':
        data_copy_clean[col] = data_copy_clean[col].astype(str)
    # Las tipo object se quedan igual

# Paso 4: Aplicar dummies
df_dummies = pd.get_dummies(data_copy_clean[variables_dummificar], drop_first=True)

# Paso 5: Conservar variables numéricas y la variable dependiente
df_no_dummies = data_copy_clean[variables_conservar]

# Paso 6: Concatenar todo
data_final = pd.concat([df_no_dummies, df_dummies], axis=1).reset_index(drop=True)

# Paso 7: Asegurar que la variable objetivo sea binaria
data_final['pob_objetivo'] = (data_final['pob_objetivo'] == 1.0).astype(int)

# Paso 8: Asegurar que todo lo que quedó como bool sea 0/1
bool_cols = data_final.select_dtypes(include='bool').columns
data_final[bool_cols] = data_final[bool_cols].astype(int)

# Verificación
print(data_final.shape)
print(data_final.dtypes.value_counts())
print(data_final.head())




(20448, 573)
int64      387
int8       183
int32        2
float64      1
Name: count, dtype: int64
   p1_1  p1_2  p1_3  p2_1  p8_4  edad_v  ingreso_mensual  gasto_mensual  \
0     2     5     2     4     9      51              NaN           1500   
1     3     5     1     5     8      41          10750.0           5000   
2     2     5     1     4     6      41           6000.0           4000   
3     1     5     2     1     0      73              NaN           2400   
4     2     5     1     3    10      50              NaN              0   

   fac_ele  pob_objetivo  ...  p9_2_4.0  p9_2_5.0  p9_2_6.0  p9_2_7.0  \
0     1299             0  ...         0         0         0         0   
1     1948             0  ...         0         0         0         0   
2     1299             0  ...         0         0         0         0   
3      649             0  ...         0         0         0         0   
4     1299             0  ...         0         0         0         0   

   p9_2_nan

In [49]:
from IPython.display import display
display(data_final)

,p1_1,p1_2,p1_3,p2_1,p8_4,edad_v,ingreso_mensual,gasto_mensual,fac_ele,pob_objetivo,...,p9_2_4.0,p9_2_5.0,p9_2_6.0,p9_2_7.0,p9_2_nan,p10_2_2.0,p10_2_9.0,p10_2_nan,p10_3_2.0,p10_3_nan
0,2,5,2,4,9,51,NaN,1500,1299,0,...,0,0,0,0,0,0,0,0,1,0
1,3,5,1,5,8,41,10750.0,5000,1948,0,...,0,0,0,0,0,0,0,1,0,1
2,2,5,1,4,6,41,6000.0,4000,1299,0,...,0,0,0,0,0,0,0,0,1,0
3,1,5,2,1,0,73,NaN,2400,649,0,...,0,0,0,0,1,0,0,1,0,1
4,2,5,1,3,10,50,NaN,0,1299,0,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20443,1,3,1,2,9,59,NaN,2000,1685,0,...,0,0,0,0,0,0,0,1,0,1
20444,3,5,1,4,0,19,NaN,2000,2527,0,...,0,0,1,0,0,0,0,1,0,1
20445,2,6,1,3,8,54,4800.0,1200,1685,0,...,0,0,0,0,0,0,1,0,0,0
20446,5,8,2,7,3,38,6450.0,4000,5897,0,...,0,0,0,0,0,0,0,1,0,1


In [51]:
# Reemplazar los NA en ingreso_mensual con 0
data_final["ingreso_mensual"] = data_final["ingreso_mensual"].fillna(0)

#sumar 0.01 a ingreso_mensual para evitar problemas de logaritmo
data_final["ingreso_mensual"] = data_final["ingreso_mensual"] + 0.01

#reemplazar la variable ingreso_mensual por su logaritmo
data_final["ingreso_mensual"] = np.log(data_final["ingreso_mensual"])

#repetir proceso para gasto_mensual
data_final["gasto_mensual"] = data_final["gasto_mensual"] + 0.01

data_final["gasto_mensual"] = np.log(data_final["gasto_mensual"])

In [56]:

# Crear resumen de variables
resumen = []

for col in data_final.columns:
    tipo = data_final[col].dtype
    n_unicos = data_final[col].nunique()

    if np.issubdtype(tipo, np.number):
        minimo = data_final[col].min()
        maximo = data_final[col].max()
    else:
        minimo = maximo = "N/A"

    resumen.append({
        "variable": col,
        "tipo": str(tipo),
        "n_valores_unicos": n_unicos,
        "min": minimo,
        "max": maximo
    })

# Convertir a DataFrame
df_resumen = pd.DataFrame(resumen)

# Obtener ruta actual y definir nombre del archivo
ruta_actual = os.getcwd()
ruta_salida = os.path.join(ruta_actual, "final.txt")

# Guardar en archivo .txt con tabuladores
df_resumen.to_csv(ruta_salida, sep='\t', index=False)

print(f"Resumen exportado correctamente a: {ruta_salida}")



Resumen exportado correctamente a: c:\SEM 8\Bloque dos & materias final\Proyecto BeWay\tablas & codigos\python\final.txt


In [62]:

# Paso 1: Separar y (objetivo) y X (predictoras)
y = data_final['pob_objetivo']
X = data_final.drop(columns=['pob_objetivo'])

# Paso 2: Clasificar variables

# Variables a escalar manualmente (detectadas por nombre y rango en final.txt)
continuas = ['edad_v', 'ingreso_mensual', 'gasto_mensual']

# Detectar variables categóricas codificadas como numéricas (<= 10 valores únicos, valores enteros no binarios)
categoricas_codificadas = [
    col for col in X.columns
    if X[col].dtype in ['int8', 'int16', 'int32', 'int64']
    and X[col].nunique(dropna=True) <= 32
    and not set(X[col].dropna().unique()).issubset({0, 1})
    and col not in continuas
]

# Convertirlas a string para tratarlas como categóricas en el encoder
X[categoricas_codificadas] = X[categoricas_codificadas].astype(str)

# Detectar columnas binarias (0/1) o ya dummies → no se tocan
binarias = [
    col for col in X.columns
    if set(X[col].dropna().unique()).issubset({0, 1})
    and col not in continuas
]

# Verificación de grupos
print("▶️ Variables continuas a escalar:", continuas)
print("▶️ Variables categóricas a dummificar:", categoricas_codificadas)
print("▶️ Variables binarias que se dejan tal cual:", binarias)

# Paso 3: Construir el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), continuas),
        ('dummies', OneHotEncoder(drop='first', sparse_output=False), categoricas_codificadas)
    ],
    remainder='passthrough'  # deja binarias como están
)

# Paso 4: Transformar X
X_preparado = preprocessor.fit_transform(X)

# Obtener todos los nombres de variables transformadas
feature_names = preprocessor.get_feature_names_out()


# Crear DataFrame final listo para modelar
X_modelo = pd.DataFrame(X_preparado, columns=feature_names)

# Verificación
print("\n✅ Matriz X_modelo lista. Dimensiones:", X_modelo.shape)
print(X_modelo.head())


▶️ Variables continuas a escalar: ['edad_v', 'ingreso_mensual', 'gasto_mensual']
▶️ Variables categóricas a dummificar: ['p1_1', 'p1_2', 'p1_3', 'p2_1', 'p8_4', 'p1_4_01', 'p1_4_02', 'p1_4_03', 'p1_4_04', 'p1_4_05', 'p1_4_06', 'p1_4_07', 'p1_4_08', 'p1_4_09', 'p1_4_10', 'p1_5', 'p2_2', 'ent', 'tloc', 'region', 'p4_1_1', 'p4_1_2', 'p4_1_3', 'p4_1_4', 'p4_1_5', 'p4_1_6', 'p4_1_7', 'p4_1_8', 'p4_1_9', 'filtro_s4_1', 'p4_10_1', 'p4_10_2', 'p4_10_3', 'p4_10_4', 'p4_10_5', 'p4_10_6', 'sexo', 'p5_1', 'p5_2_1', 'p5_2_2', 'p5_2_3', 'p5_2_4', 'p5_2_5', 'p5_2_6', 'p5_2_7', 'p5_2_8', 'p5_3', 'p5_5', 'p5_10_1', 'p5_11', 'filtro_s5_2', 'filtro_s5_3', 'p5_14', 'filtro_s5_4', 'p6_1_1', 'p6_1_2', 'p6_1_3', 'p6_1_4', 'p6_1_5', 'p6_1_6', 'p6_2_01', 'p6_2_02', 'p6_2_03', 'p6_2_04', 'p6_2_05', 'p6_2_06', 'p6_2_07', 'p6_2_08', 'p6_2_09', 'p6_2_10', 'filtro_s6_1', 'p6_5_1', 'p6_5_2', 'p6_5_3', 'p6_5_4', 'p6_5_5', 'p6_6_1', 'p6_6_2', 'p6_6_3', 'p6_6_4', 'p6_6_5', 'p6_6_6', 'p6_6_7', 'p6_6_8', 'p6_6_9', 'filtr

In [65]:
# Contar cuántas variables tiene el DataFrame
num_variables = X_modelo.shape[1]
print(f"El DataFrame tiene {num_variables} variables.")

El DataFrame tiene 832 variables.


In [67]:
# Modelo: Lasso logit con validación cruzada (5 folds)

# Definir rango manual de C más amplio (log10(C) desde -4 hasta 0.5 → C desde 0.0001 a 3.16)
C_values = np.logspace(-4, 0.5, 100)

lasso_logit_cv = LogisticRegressionCV(
    Cs=C_values,  #valores de C (1/λ)
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    penalty='l1',
    solver='saga',
    scoring='neg_log_loss',
    max_iter=5000,
    random_state=42,
    n_jobs=-1
)

# Entrenamiento
lasso_logit_cv.fit(X_modelo, y)

# Coeficientes del modelo
coefs = lasso_logit_cv.coef_.flatten()
selected = coefs != 0

# Reporte
print("🔁 Con penalización reforzada:")
print("✅ Variables seleccionadas:", selected.sum())
print("📌 Mejor C (1/lambda):", lasso_logit_cv.C_[0])

vars_seleccionadas = X_modelo.columns[selected].tolist()
for var in vars_seleccionadas:
    print("-", var)

🔁 Con penalización reforzada:
✅ Variables seleccionadas: 822
📌 Mejor C (1/lambda): 3.1622776601683795
- scale__edad_v
- scale__ingreso_mensual
- scale__gasto_mensual
- dummies__p1_1_2
- dummies__p1_1_3
- dummies__p1_1_4
- dummies__p1_1_5
- dummies__p1_1_6
- dummies__p1_1_7
- dummies__p1_2_10
- dummies__p1_2_11
- dummies__p1_2_12
- dummies__p1_2_13
- dummies__p1_2_14
- dummies__p1_2_18
- dummies__p1_2_2
- dummies__p1_2_3
- dummies__p1_2_4
- dummies__p1_2_5
- dummies__p1_2_6
- dummies__p1_2_7
- dummies__p1_2_8
- dummies__p1_2_9
- dummies__p1_3_1
- dummies__p1_3_2
- dummies__p1_3_3
- dummies__p1_3_4
- dummies__p1_3_5
- dummies__p1_3_6
- dummies__p1_3_7
- dummies__p1_3_8
- dummies__p2_1_10
- dummies__p2_1_11
- dummies__p2_1_13
- dummies__p2_1_14
- dummies__p2_1_15
- dummies__p2_1_2
- dummies__p2_1_3
- dummies__p2_1_4
- dummies__p2_1_5
- dummies__p2_1_6
- dummies__p2_1_7
- dummies__p2_1_8
- dummies__p2_1_9
- dummies__p8_4_1
- dummies__p8_4_10
- dummies__p8_4_2
- dummies__p8_4_3
- dummies__p

In [ ]:
#EL LASSO NO CASTIGA SUFICIENTES VARIABLES

#POST-DOUBLE LASSO 

In [69]:


# Paso 1: Lasso para predecir y
logit_lasso = LogisticRegressionCV(
    penalty='l1',
    solver='saga',
    Cs=np.logspace(-4, 0.5, 100),
    scoring='neg_log_loss',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    max_iter=5000,
    random_state=42,
    n_jobs=-1
)

logit_lasso.fit(X_modelo, y)
selected_y = np.flatnonzero(logit_lasso.coef_.flatten() != 0)
vars_y = set(X_modelo.columns[selected_y])
print(f"🔎 Paso 1: {len(vars_y)} variables seleccionadas para explicar y")


🔎 Paso 1: 822 variables seleccionadas para explicar y


In [ ]:
# Paso 2: Lasso para cada X_j ~ X_{-j}
vars_x = set()

for j, col in enumerate(X_modelo.columns):
    X_j = X_modelo.drop(columns=[col])
    y_j = X_modelo[col]

    # Lasso clásico (modelo de regresión lineal)
    lasso = LassoCV(cv=5, alphas=np.logspace(-4, 0, 100), max_iter=5000, random_state=42)
    lasso.fit(X_j, y_j)

    # Seleccionadas para explicar esta X_j
    selected_j = X_j.columns[lasso.coef_ != 0]
    vars_x.update(selected_j)

print(f"🔎 Paso 2: {len(vars_x)} variables seleccionadas para explicar al menos una X_j")


In [ ]:
# Unión de variables de pasos 1 y 2
vars_final = sorted(vars_y.union(vars_x))
print(f"✅ Paso 3: {len(vars_final)} variables finales para el modelo logit")


In [ ]:
# Modelo logit final sin penalización
X_logit = sm.add_constant(X_modelo[vars_final])
modelo_final = sm.Logit(y, X_logit).fit()

# Resultados
print(modelo_final.summary())
